In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.initializers import HeNormal, RandomNormal
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import matplotlib
from sklearn.model_selection import train_test_split

In [2]:
print(matplotlib.__version__)
print(tf.__version__)

3.7.1
2.10.0


In [3]:
def load_image_paths_by_class(dataset_path):
    
    #สร้าง dictionary 
    class_image_paths = {}

    for class_name in os.listdir(dataset_path): #os.listdir ดึงรายชื่อไฟล์/โฟเดอร์ใน paht
        class_dir = os.path.join(dataset_path, class_name)#os.path.join = images\photo.jpg
        #print(class_dir)
        if os.path.isdir(class_dir):#ตรวจสอบclass_dir เป็นโฟลเดอร์ถ้าเป็นโฟลเดอร์ จะเข้าไปทำงานต่อ
            image_files = [ 
                os.path.join(class_dir, img) #
                for img in os.listdir(class_dir) # ดึงรายชื่อ
                if img.lower().endswith(('.jpg', '.jpeg', '.png')) #
            ]
            if len(image_files) >= 20:  # ขั้นต่ำ 3 ภาพผป
                class_image_paths[class_name] = image_files # สร้าง dictionary

    return class_image_paths


In [4]:
#dataset_path = r'C:\InceptionNet\mediamdata' 
dataset_path3 = r"D:\InceptionNet\GLint360K\glint360k_3"
dic = load_image_paths_by_class(dataset_path3)
print(len(dic))
#print(dictm)


11023


In [5]:
classes = list(dic.keys())
#print(classes)
#print(len(classes))

In [6]:
train_classes, val_classes = train_test_split(classes, test_size=0.2, random_state=42)

print(f"จำนวนคลาสสำหรับ Training: {len(train_classes)}")
print(f"จำนวนคลาสสำหรับ Validation: {len(val_classes)}")


จำนวนคลาสสำหรับ Training: 8818
จำนวนคลาสสำหรับ Validation: 2205


In [7]:
train_image_dict1 = {
    class_name: dic[class_name]
    for class_name in train_classes
}

# สร้าง dictionary สำหรับ Validation
val_image_dict1 = {
    class_name: dic[class_name]
    for class_name in val_classes
}
print(f"ขนาดของ Dictionary สำหรับ Training: {len(train_image_dict1)}")
print(f"ขนาดของ Dictionary สำหรับ Validation: {len(val_image_dict1)}")



ขนาดของ Dictionary สำหรับ Training: 8818
ขนาดของ Dictionary สำหรับ Validation: 2205


In [8]:
#print("\n--- รายชื่อคลาสสำหรับ Training ---")
#for class_name in train_classes:
    #print(class_name)

# ---
# ปริ้นชื่อคลาสสำหรับ Validation
print("\n--- รายชื่อคลาสสำหรับ Validation ---")
for class_name in val_classes:
    print(class_name)


--- รายชื่อคลาสสำหรับ Validation ---
109337
256904
98372
282304
87882
169735
220290
108233
88308
114954
150320
232249
235900
99213
107866
148997
145581
326282
54954
196803
231646
160264
128659
185806
63188
358334
28781
301834
67526
183485
215932
199098
160523
29544
141279
184612
343814
314639
112465
111742
90325
74100
93370
354014
260064
94862
347907
2680
255115
210198
233912
74987
292
28330
113587
210560
148194
62405
148019
122870
238582
114201
33816
8071
75322
85173
43382
270330
238140
27544
84262
39532
258863
130796
286409
179613
110594
250149
49947
209426
26786
20359
23506
120841
126679
252394
192265
105881
324373
16120
308780
98231
259461
37372
180645
306274
122146
271432
101022
315543
125592
14059
255162
187935
173374
287073
232459
46472
189602
93498
340547
191538
196907
262027
161495
296378
102884
218197
86212
290841
188571
328132
224706
149239
243695
224459
34125
30720
252225
60874
9624
159312
36860
216310
54946
37598
90629
56476
80843
6612
31332
170911
154364
241497
342001
30

In [9]:
def loadimage(image_path):
    try:
        img = tf.io.read_file(image_path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, (224, 224))
        img = tf.cast(img, tf.float32)
        img = img / 255.0  # Normalization
        img = tf.clip_by_value(img, 0.0, 1.0)#img = tf.where(tf.math.is_finite(img), img, tf.zeros_like(img)) # แทนที่ค่า NaN หรือ Inf ด้วย 0
        img.set_shape((224, 224, 3))
        return img
    except:
        print(f"Error loading image {image_path}. Returning a zero tensor.")
        return tf.zeros((224, 224, 3), dtype=tf.float32)
    
    

In [10]:
import random
import numpy as np
from tensorflow.keras.preprocessing import image

def triplet_generator(model, image_dict, num_triplets_per_epoch, margin=0.2):
    
    classes = list(image_dict.keys())

    for _ in range(num_triplets_per_epoch):
        # -------------------------------
        # 1️⃣ สุ่ม anchor และ positive จากคลาสเดียวกัน
        # -------------------------------
        anchor_class = random.choice(classes)
        if len(image_dict[anchor_class]) < 2:
            continue

        a_path, p_path = random.sample(image_dict[anchor_class], 2)

        # โหลดภาพและทำ normalization
        def load_img(path):
            img = image.load_img(path, target_size=(224, 224))
            arr = image.img_to_array(img) / 255.0
            return arr

        anchor_img = loadimage(a_path)
        positive_img = load_img(p_path)

        # สร้าง embeddings ของ anchor, positive
        a_emb = model.predict(np.expand_dims(anchor_img, axis=0))
        p_emb = model.predict(np.expand_dims(positive_img, axis=0))

        # -------------------------------
        # 2️⃣ หา Negative จากคลาสอื่น
        # -------------------------------
        neg_classes = [c for c in classes if c != anchor_class]
        if not neg_classes:
            continue

        # รวมรูปจากหลายคลาสอื่น ๆ
        neg_paths = [random.choice(image_dict[c]) for c in random.sample(neg_classes, min(len(neg_classes), 10))]
        neg_imgs = np.array([load_img(p) for p in neg_paths])

        # สร้าง embeddings ของ negatives ทั้งหมด
        neg_embs = model.predict(neg_imgs)

        # -------------------------------
        # 3️⃣ เลือก Semi-Hard Negative
        # -------------------------------
        d_ap = np.linalg.norm(a_emb - p_emb)  # distance ระหว่าง anchor-positive
        distances = np.linalg.norm(neg_embs - a_emb, axis=1)

        # เงื่อนไข semi-hard: d_ap < d_an < d_ap + margin
        semi_hard_mask = (distances > d_ap) & (distances < d_ap + margin)
        semi_hard_indices = np.where(semi_hard_mask)[0]

        if len(semi_hard_indices) > 0:
            chosen_idx = random.choice(semi_hard_indices)
        else:
            # ถ้าไม่มี semi-hard เลือก negative ที่ใกล้ที่สุด (hard)
            chosen_idx = np.argmin(distances)

        n_path = neg_paths[chosen_idx]
        n_emb = neg_embs[chosen_idx]

        # -------------------------------
        # 4️⃣ คืนค่า Path ของ triplet (หรือจะส่งเป็นภาพ/embedding ก็ได้)
        # -------------------------------
        yield (a_path, p_path, n_path)


In [11]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Lambda, Input,Dropout

In [12]:
def QQQ():
    input_shape = (224, 224, 3)

    base_model = ResNet50(
        include_top=False,
        weights='imagenet',
        input_shape=input_shape
    )
    for layer in base_model.layers:
        layer.trainable = False
        
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128)(x)
    embedding = Lambda(lambda x: tf.math.l2_normalize(x, axis=1))(x)
    
    # สร้างโมเดลและ return
    model = Model(inputs=base_model.input, outputs=embedding)
    
    return model

In [13]:
model = QQQ()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [14]:
def builddataset1(image_dict, num_triplets, batch_size, embedding_model):
    dataset = tf.data.Dataset.from_generator(
        lambda: triplet_generator(image_dict, num_triplets, embedding_model),
        output_signature=(
            tf.TensorSpec(shape=(), dtype=tf.string),
            tf.TensorSpec(shape=(), dtype=tf.string),
            tf.TensorSpec(shape=(), dtype=tf.string)
        )
    )

    dataset = dataset.map(
        lambda a, p, n: (loadimage(a), loadimage(p), loadimage(n)),
        num_parallel_calls=tf.data.AUTOTUNE
    )

    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset
triple = 50000
BATCH_SIZE = 4

train_dataset = builddataset1(train_image_dict1, triple, BATCH_SIZE, embedding_model=model)
val_dataset   = builddataset1(val_image_dict1, int(triple*0.1), BATCH_SIZE, embedding_model=model)


In [15]:
l = 0

# ทดสอบ
for a, p, n in train_dataset.take(5):
    print("Anchor batch shape:", a.shape)
    print("Positive batch shape:", p.shape)
    print("Negative batch shape:", n.shape)
    print("ลูปที่:", a)
    #l = l + 1Q
    print("ลูปที่:", l)

UnknownError: {{function_node __wrapped__IteratorGetNext_output_types_3_device_/job:localhost/replica:0/task:0/device:CPU:0}} AttributeError: 'int' object has no attribute 'keys'
Traceback (most recent call last):

  File "d:\final_facenet\env\lib\site-packages\tensorflow\python\ops\script_ops.py", line 271, in __call__
    ret = func(*args)

  File "d:\final_facenet\env\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "d:\final_facenet\env\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py", line 1035, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "C:\Users\maxx5\AppData\Local\Temp\ipykernel_34112\297625382.py", line 7, in triplet_generator
    classes = list(image_dict.keys())

AttributeError: 'int' object has no attribute 'keys'


	 [[{{node PyFunc}}]] [Op:IteratorGetNext]

In [ ]:
for i, layer in enumerate(model.layers):
    print(i, layer.name, layer.trainable)


0 input_5 False
1 conv1_pad False
2 conv1_conv False
3 conv1_bn False
4 conv1_relu False
5 pool1_pad False
6 pool1_pool False
7 conv2_block1_1_conv False
8 conv2_block1_1_bn False
9 conv2_block1_1_relu False
10 conv2_block1_2_conv False
11 conv2_block1_2_bn False
12 conv2_block1_2_relu False
13 conv2_block1_0_conv False
14 conv2_block1_3_conv False
15 conv2_block1_0_bn False
16 conv2_block1_3_bn False
17 conv2_block1_add False
18 conv2_block1_out False
19 conv2_block2_1_conv False
20 conv2_block2_1_bn False
21 conv2_block2_1_relu False
22 conv2_block2_2_conv False
23 conv2_block2_2_bn False
24 conv2_block2_2_relu False
25 conv2_block2_3_conv False
26 conv2_block2_3_bn False
27 conv2_block2_add False
28 conv2_block2_out False
29 conv2_block3_1_conv False
30 conv2_block3_1_bn False
31 conv2_block3_1_relu False
32 conv2_block3_2_conv False
33 conv2_block3_2_bn False
34 conv2_block3_2_relu False
35 conv2_block3_3_conv False
36 conv2_block3_3_bn False
37 conv2_block3_add False
38 conv2_bloc

In [16]:

def model3():
    input_shape = (224, 224, 3)

    base_model = ResNet50(
        include_top=False,
        weights='imagenet',
        input_shape=input_shape
    )
    
    for layer in base_model.layers:
        layer.trainable = False
    
    for layer in base_model.layers[-30:]:
        layer.trainable = True

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128)(x)
    embedding = Lambda(lambda x: tf.math.l2_normalize(x, axis=1))(x)
    
    # สร้างโมเดลและ return
    model = Model(inputs=base_model.input, outputs=embedding)
    
    return model

In [12]:
def model_drop():
    input_shape = (224, 224, 3)

    base_model = ResNet50(
        include_top=False,
        weights='imagenet',
        input_shape=input_shape
    )
    
    for layer in base_model.layers:
        layer.trainable = False
    
    for layer in base_model.layers[-30:]:
        layer.trainable = True

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x) 
    x = Dense(128)(x)
    embedding = Lambda(lambda x: tf.math.l2_normalize(x, axis=1))(x)
    
    model = Model(inputs=base_model.input, outputs=embedding)
    
    return model


In [13]:
model = model_drop()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [14]:
for i, layer in enumerate(model.layers):
    print(i, layer.name, layer.trainable)

0 input_1 False
1 conv1_pad False
2 conv1_conv False
3 conv1_bn False
4 conv1_relu False
5 pool1_pad False
6 pool1_pool False
7 conv2_block1_1_conv False
8 conv2_block1_1_bn False
9 conv2_block1_1_relu False
10 conv2_block1_2_conv False
11 conv2_block1_2_bn False
12 conv2_block1_2_relu False
13 conv2_block1_0_conv False
14 conv2_block1_3_conv False
15 conv2_block1_0_bn False
16 conv2_block1_3_bn False
17 conv2_block1_add False
18 conv2_block1_out False
19 conv2_block2_1_conv False
20 conv2_block2_1_bn False
21 conv2_block2_1_relu False
22 conv2_block2_2_conv False
23 conv2_block2_2_bn False
24 conv2_block2_2_relu False
25 conv2_block2_3_conv False
26 conv2_block2_3_bn False
27 conv2_block2_add False
28 conv2_block2_out False
29 conv2_block3_1_conv False
30 conv2_block3_1_bn False
31 conv2_block3_1_relu False
32 conv2_block3_2_conv False
33 conv2_block3_2_bn False
34 conv2_block3_2_relu False
35 conv2_block3_3_conv False
36 conv2_block3_3_bn False
37 conv2_block3_add False
38 conv2_bloc

In [15]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

In [16]:
def triplet_loss(anchor, positive, negative, alpha=0.2):
    epsilon = 1e-8
    
    # คำนวณระยะห่างแบบ Euclidean squared
    pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis=1)
    neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis=1)
    
    # เพิ่ม epsilon และใช้ tf.clip_by_value เพื่อป้องกันค่าผิดปกติ
    pos_dist = tf.clip_by_value(pos_dist, epsilon, 10.0)
    neg_dist = tf.clip_by_value(neg_dist, epsilon, 10.0)
    
    # คำนวณ basic loss
    basic_loss = pos_dist - neg_dist + alpha
    
    # ใช้ tf.nn.relu แทน tf.maximum
    loss = tf.reduce_mean(tf.nn.relu(basic_loss))
    
    # ตรวจสอบ NaN
    loss = tf.where(tf.math.is_finite(loss), loss, tf.constant(0.0))
    
    return loss


In [21]:
import tensorflow as tf

def losshard_mining(anchor, positive, negative, alpha=0.2):
    """
    Computes Triplet Loss with Hard Triplet Mining.
    """
    # 1. Calculate squared Euclidean distances
    pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis=1)
    neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis=1)

    # 2. Find all valid triplets in the batch
    all_triplets = pos_dist - neg_dist + alpha
    
    # 3. Filter out the easy triplets (loss <= 0)
    hard_triplets = tf.where(all_triplets > 0, all_triplets, tf.zeros_like(all_triplets))

    # 4. Take the mean of the hard triplets
    # Add a small value to the denominator to avoid division by zero
    num_hard_triplets = tf.reduce_sum(tf.cast(hard_triplets > 0, dtype=tf.float32))
    
    # Check if there are any hard triplets. If not, the loss is 0
    if num_hard_triplets > 0:
        loss = tf.reduce_sum(hard_triplets) / num_hard_triplets
    else:
        loss = tf.constant(0.0)

    # Return the loss
    return loss

In [17]:
@tf.function
def triplet_loss_val(anchor, positive, negative, alpha=0.2):
    epsilon = 1e-8
    
    # คำนวณระยะห่างแบบ Euclidean squared
    pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis=1)
    neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis=1)
    
    # เพิ่ม epsilon และใช้ tf.clip_by_value เพื่อป้องกันค่าผิดปกติ
    pos_dist = tf.clip_by_value(pos_dist, epsilon, 10.0)
    neg_dist = tf.clip_by_value(neg_dist, epsilon, 10.0)
    
    # คำนวณ basic loss
    basic_loss = pos_dist - neg_dist + alpha
    
    # ใช้ tf.nn.relu แทน tf.maximum
    loss = tf.reduce_mean(tf.nn.relu(basic_loss))
    
    # ตรวจสอบ NaN
    loss = tf.where(tf.math.is_finite(loss), loss, tf.constant(0.0))
    
    return loss

In [18]:
@tf.function
def train_step(anchor, positive, negative):
    with tf.GradientTape() as tape:
        anchor_embedding = model(anchor, training=True)
        positive_embedding = model(positive, training=True)
        negative_embedding = model(negative, training=True)
        loss = triplet_loss(anchor_embedding, positive_embedding, negative_embedding)
    
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

In [ ]:
EPOCHS = 50
print("--- เริ่มการฝึกโมเดล ---")
train_loss_history = []
val_loss_history = []
for epoch in range(EPOCHS):
    total_loss = 0.0
    num_batches = 0
    
    # วนลูปใน training dataset
    print(f"\n--- Epoch {epoch+1}/{EPOCHS} ---")
    for anchor, positive, negative in train_dataset:
        loss = train_step(anchor, positive, negative)
        total_loss += loss
        num_batches += 1
        
        # ปริ้นท์
        if num_batches % 5000 == 0:
            print(f"Batch {num_batches}, Training Loss: {loss.numpy():.4f}")
    
    avg_loss = total_loss / num_batches if num_batches > 0 else 0
    print(f"\nสรุป Epoch {epoch+1}/{EPOCHS}, Training Loss เฉลี่ย: {avg_loss.numpy():.4f}")
    train_loss_history.append(avg_loss)
    # Optional: ตรวจสอบ Validation Loss ในแต่ละ Epoch
    val_total_loss = 0.0
    val_num_batches = 0
    
    # วนลูปใน validation dataset
    print("\n--- เริ่ม Validation ---")
    for anchor, positive, negative in val_dataset:
        anchor_embedding = model(anchor, training=False)
        positive_embedding = model(positive, training=False)
        negative_embedding = model(negative, training=False)
        val_loss = triplet_loss(anchor_embedding, positive_embedding, negative_embedding)
        val_total_loss += val_loss
        val_num_batches += 1
        
        # ปริ้นท์ Val
        if val_num_batches % 500 == 0:
             print(f"Validation Batch {val_num_batches}, Validation Loss: {val_loss.numpy():.4f}")
             
    val_avg_loss = val_total_loss / val_num_batches if val_num_batches > 0 else 0
    print(f"สรุป Validation Loss เฉลี่ย: {val_avg_loss.numpy():.4f}")
    val_loss_history.append(val_avg_loss)
    nameModel = f"modelR{epoch+1}.h5"
    model.save(nameModel)

plt.figure(figsize=(10, 6))
plt.plot(range(1, EPOCHS + 1), train_loss_history, 'o-', label='Training Loss',color='red')
plt.plot(range(1, EPOCHS + 1), val_loss_history, 'x-', label='Validation Loss',color="#00FF2A")
plt.title('Training vs. Validation Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

print("\n--- การฝึกโมเดลเสร็จสิ้น ---")
model.save("2M.h5")


--- เริ่มการฝึกโมเดล ---

--- Epoch 1/50 ---
Batch 5000, Training Loss: 0.4451
Batch 10000, Training Loss: 0.2398

สรุป Epoch 1/50, Training Loss เฉลี่ย: 0.1863

--- เริ่ม Validation ---
Validation Batch 500, Validation Loss: 0.1845
Validation Batch 1000, Validation Loss: 0.2856
สรุป Validation Loss เฉลี่ย: 0.1785

--- Epoch 2/50 ---
Batch 5000, Training Loss: 0.2171
Batch 10000, Training Loss: 0.4500

สรุป Epoch 2/50, Training Loss เฉลี่ย: 0.1698

--- เริ่ม Validation ---
Validation Batch 500, Validation Loss: 0.0926
Validation Batch 1000, Validation Loss: 0.0457
สรุป Validation Loss เฉลี่ย: 0.1710

--- Epoch 3/50 ---
Batch 5000, Training Loss: 0.3533
Batch 10000, Training Loss: 0.2443

สรุป Epoch 3/50, Training Loss เฉลี่ย: 0.1600

--- เริ่ม Validation ---
Validation Batch 500, Validation Loss: 0.1771
Validation Batch 1000, Validation Loss: 0.0423
สรุป Validation Loss เฉลี่ย: 0.1477

--- Epoch 4/50 ---
Batch 5000, Training Loss: 0.1685
Batch 10000, Training Loss: 0.1606

สรุป Epoch 4

In [ ]:
import tensorflow as tf

def hard_mining(anchor, positive, negative, alpha=0.2):
    pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis=1)
    neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis=1)
    hardest_pos_dist = tf.reduce_max(pos_dist)
    hardest_neg_dist = tf.reduce_min(neg_dist)
    basic_loss = hardest_pos_dist - hardest_neg_dist + alpha
    loss = tf.nn.relu(basic_loss)
    return loss